<a href="https://colab.research.google.com/github/diputs03/AI-Studies/blob/main/From-tensorflow-mnist-tutorial/dynamic_architect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicNet(nn.Module):
    def __init__(self, add_intermediate=lambda: False):
        super(DynamicNet, self).__init__()
        self.add_intermediate = add_intermediate
        # Common input layer: from 784 to 256 neurons
        self.input_layer = nn.Linear(784, 256)

        # Optional intermediate layer: from 256 to 256 neurons
        if self.add_intermediate:
            self.intermediate_layer = nn.Linear(256, 256)

        # Output layer: from 256 to 10 neurons
        self.output_layer = nn.Linear(256, 10)

    def forward(self, x):
        # Flatten input if necessary (assuming x is [batch_size, 28, 28])
        x = x.view(x.size(0), -1)
        x = F.relu(self.input_layer(x))
        if self.add_intermediate:
            x = F.relu(self.intermediate_layer(x))
        x = self.output_layer(x)
        return x

# Example usage:

# Condition: whether to add an intermediate layer
def insert_extra_layer():
  return True  # Change this flag as needed

# Create the model with the desired architecture
model = DynamicNet(add_intermediate=insert_extra_layer)

# Print model structure
print(model)

# Dummy input for testing (e.g., a batch of 64 MNIST images)
dummy_input = torch.randn(64, 1, 28, 28)
output = model(dummy_input)
print("Output shape:", output.shape)


DynamicNet(
  (input_layer): Linear(in_features=784, out_features=256, bias=True)
  (intermediate_layer): Linear(in_features=256, out_features=256, bias=True)
  (output_layer): Linear(in_features=256, out_features=10, bias=True)
)
Output shape: torch.Size([64, 10])


In [4]:
class FlexibleNet(nn.Module):
    def __init__(self):
        super(FlexibleNet, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(784, 256), nn.Linear(256, 10)])

    def forward(self, x):
        x = x.view(x.size(0), -1)
        # Dynamically use each layer
        for layer in self.layers:
            x = F.relu(layer(x))
        return x

    def insert_layer(self, index, layer):
        self.layers.insert(index, layer)

# Example: Inserting a new layer between the first and second layers
net = FlexibleNet()
print("Before inserting:", net)
net.insert_layer(1, nn.Linear(256, 256))
print("After inserting:", net)


Before inserting: FlexibleNet(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=10, bias=True)
  )
)
After inserting: FlexibleNet(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MaskedLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__()
        # Standard linear layer parameters.
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        # Create a mask of ones (initially, all connections are enabled)
        self.register_buffer('mask', torch.ones(out_features, in_features))

    def forward(self, x):
        # Apply the mask to the weight matrix.
        masked_weight = self.linear.weight * self.mask
        # Use the masked weight in the linear operation.
        return F.linear(x, masked_weight, self.linear.bias)

    def update_mask(self, new_mask):
        """
        Update the connectivity mask.

        Args:
            new_mask (torch.Tensor): A tensor with shape (out_features, in_features)
                                     containing 0s and 1s.
        """
        self.mask.copy_(new_mask)

# Example usage:
# Create a masked linear layer for a 784->10 mapping.
masked_layer = MaskedLinear(784, 10)

# Suppose you want to disable the connection from input neuron 5 to output neuron 3:
# Get the current mask, update the specified entry, and then update the mask.
current_mask = masked_layer.mask.clone()
current_mask[3, 5] = 0  # 0 disables the connection.
masked_layer.update_mask(current_mask)

# Now, in a model, you can use MaskedLinear in place of nn.Linear.
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc = MaskedLinear(784, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Dummy data for demonstration (e.g., one MNIST image flattened)
dummy_input = torch.randn(1, 1, 28, 28)
model = CustomModel()
print(model)
output = model(dummy_input)
print("Output:", output)


CustomModel(
  (fc): MaskedLinear(
    (linear): Linear(in_features=784, out_features=10, bias=True)
  )
)
Output: tensor([[-0.2138, -0.5612, -0.0726,  0.0615, -0.9300,  0.4176,  0.3547, -0.2046,
          0.5302,  0.1620]], grad_fn=<AddmmBackward0>)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ExtraNeuronModel(nn.Module):
    def __init__(self, add_extra=False):
        """
        Args:
            add_extra (bool): Whether to include an extra neuron in the network.
        """
        super(ExtraNeuronModel, self).__init__()
        self.add_extra = add_extra

        # Base pathway: from 784 inputs to 10 outputs.
        self.base_layer = nn.Linear(784, 10)

        if self.add_extra:
            # Extra neuron computed from the same input.
            self.extra_neuron = nn.Linear(784, 1)
            # A combination layer that merges the original 10 outputs with the extra neuron.
            # It takes 10 + 1 = 11 features and produces 10 outputs.
            self.combination_layer = nn.Linear(11, 10)

    def forward(self, x):
        # Flatten the input (assuming x is [batch_size, 28, 28])
        x_flat = x.view(x.size(0), -1)
        base_out = self.base_layer(x_flat)  # shape: [batch_size, 10]

        if self.add_extra:
            # Compute extra neuron output
            extra_out = self.extra_neuron(x_flat)  # shape: [batch_size, 1]
            # Concatenate the base outputs with the extra neuron output
            combined = torch.cat([base_out, extra_out], dim=1)  # shape: [batch_size, 11]
            # Produce final output via the combination layer
            final_out = self.combination_layer(combined)
            return final_out
        else:
            return base_out

# Example usage:

# Create input data (e.g., a batch of 64 MNIST images of size 28x28)
dummy_input = torch.randn(64, 1, 28, 28)

# Model without the extra neuron (standard 784 -> 10)
model_standard = ExtraNeuronModel(add_extra=False)
output_standard = model_standard(dummy_input)
print("Output shape (without extra neuron):", output_standard.shape)

# Model with an extra neuron added
model_extra = ExtraNeuronModel(add_extra=True)
output_extra = model_extra(dummy_input)
print("Output shape (with extra neuron):", output_extra.shape)
print(model_extra)

Output shape (without extra neuron): torch.Size([64, 10])
Output shape (with extra neuron): torch.Size([64, 10])
ExtraNeuronModel(
  (base_layer): Linear(in_features=784, out_features=10, bias=True)
  (extra_neuron): Linear(in_features=784, out_features=1, bias=True)
  (combination_layer): Linear(in_features=11, out_features=10, bias=True)
)


In [1]:
#@title Here is mine
import numpy as np
import matplotlib
from matplotlib import pyplot

In [2]:
class Neuron:
  def relu(params, x):
    assert(len(params)!=2)
    if x>=0: return params[0]+params[1]*x
    else: return params[0]
  def sigmoid(params, x):
    return 1/(1+np.e**(-params[0]-params[1]*x))
  def linear(params, x):
    assert(len(params)!=2)
    return params[0]+params[1]*x
  def polynomial(params, x):
    ret=0
    for i in range(len(params)):
      ret+=params[i]*(x**i)
    return ret



  params=[0,0]
  next=[]
  name=''
  def __init__(self, name, next):
    self.name=name
    self.next=next
  def forward(self, next):
    self.next+=next
  def __str__(self):
    ret=str(self.name)+'('
    for n in self.next:
      ret+=str(n)
    return ret+')'

In [3]:
a=Neuron(name='a', next=[])
b=Neuron(name='b', next=[])
c=Neuron(name='c', next=[])
a.forward([b])
a.forward([c])
for n in b.next:
  print(n)
print(a)

a(b()c())


In [2]:
from sklearn.preprocessing import PolynomialFeatures

In [8]:
poly=PolynomialFeatures(degree=2)
poly.fit([[2],[3],[4],[5],[6],[7],[8],[9]], [2,3,4,5,6,7,8,9])

PolynomialFeatures()

In [10]:
poly.fit_transform([[1], [2], [3], [4]])

array([[ 1.,  1.,  1.],
       [ 1.,  2.,  4.],
       [ 1.,  3.,  9.],
       [ 1.,  4., 16.]])